In [ ]:
import pandas
import pystan
import numpy
import scipy.stats
import tensorflow

NeuronTopGeneGuidesMixtureProbs = pandas.read_csv("NeuronTopGeneGuidesMixtureProbs.txt", sep='\t', header=0)
SelfRenewalTopGeneGuidesMixtureProbs = pandas.read_csv("NeuronTopGeneGuidesMixtureProbs.txt", sep='\t', header=0)


tensorflow.Session()

with tensorflow.device('/cpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3])
# If you have gpu you can try this line to compute b with your GPU
#with tf.device('/gpu:0'):    
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2])
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

print(sess.run(c))

# Runs the op.
# Log information
options = tf.RunOptions(output_partition_graphs=True)
metadata = tf.RunMetadata()
c_val = sess.run(c, options=options, run_metadata=metadata)

print(metadata.partition_graphs)

sess.close()